Install Dependencies (if needed)

In [32]:
pip install scipy numpy pandas librosa scikit-learn tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at f:\programming ides\python\lib\site-packages\eth_abi-5.1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at f:\programming ides\python\lib\site-packages\eth_account-0.13.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at f:\programming ides\python\lib\site-packages\eth_hash-0.7.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at f:\programming ides\python\lib\site-packages\eth_typing-5.0.0b3-py3.12.egg is deprecated. pip 25.1 

Import Libraries

In [33]:
import scipy.io
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

Load dataset

In [34]:
data = scipy.io.loadmat("Newdataset.mat")

Extract data

In [35]:
X = []
y = []
label_map = {'normal': 0, 'inner': 1, 'roller': 2, 'outer': 3}

for key in data.keys():
    if key in label_map:  
        X.extend(data[key]) 
        y.extend([label_map[key]] * data[key].shape[0]) 

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



--------------------------------------------

Feature Extraction: Convert 1D signals to MFCC features

-------------------------------------------------

Convert 1D signals to MFCC features

In [36]:
def extract_mfcc(signal, sr=12000, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfccs, axis=1)  # Mean of MFCCs

X_train_mfcc = np.array([extract_mfcc(x) for x in X_train])
X_test_mfcc = np.array([extract_mfcc(x) for x in X_test])

Train Random Forest Model

In [37]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_mfcc, y_train)

RandomForestClassifier(random_state=42)

Predict & Evaluate Random Forest Model

In [38]:
y_pred_rf = rf_model.predict(X_test_mfcc)
rf_report = classification_report(y_test, y_pred_rf)
rf_cm = confusion_matrix(y_test, y_pred_rf)

--------------------- CNN Model (Spectrograms) ---------------------

In [39]:
def create_spectrogram(signal, sr=12000):
    fig, ax = plt.subplots(figsize=(2, 2))
    S = librosa.feature.melspectrogram(y=signal, sr=sr)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), sr=sr, x_axis='time', y_axis='mel', ax=ax)
    ax.axis('off')
    fig.canvas.draw()
    img = np.array(fig.canvas.renderer.buffer_rgba())[:, :, :3]
    plt.close(fig)
    return img

X_train_img = np.array([create_spectrogram(x) for x in X_train]) / 255.0
X_test_img = np.array([create_spectrogram(x) for x in X_test]) / 255.0
y_train_img = to_categorical(y_train, num_classes=len(label_map))
y_test_img = to_categorical(y_test, num_classes=len(label_map))

Train CNN Model

In [40]:
cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(X_train_img.shape[1], X_train_img.shape[2], 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_map), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_img, y_train_img, epochs=10, validation_data=(X_test_img, y_test_img), batch_size=4)


Epoch 1/10


f:\Programming IDEs\Python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 264ms/step - accuracy: 0.2659 - loss: 4.9251 - val_accuracy: 0.5417 - val_loss: 1.2564
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 251ms/step - accuracy: 0.5219 - loss: 1.2596 - val_accuracy: 0.7500 - val_loss: 0.6918
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 225ms/step - accuracy: 0.8036 - loss: 0.6010 - val_accuracy: 0.7500 - val_loss: 0.8815
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 215ms/step - accuracy: 0.7553 - loss: 0.5283 - val_accuracy: 1.0000 - val_loss: 0.0958
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 212ms/step - accuracy: 0.9622 - loss: 0.1574 - val_accuracy: 1.0000 - val_loss: 0.0296
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step - accuracy: 0.9545 - loss: 0.1011 - val_accuracy: 1.0000 - val_loss: 0.0167
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 210ms/step - accuracy: 0.9301 - loss: 0.1239 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 214ms/step - accuracy: 1.0000 - loss: 0.0383 - val_accuracy: 1.0000 - val_

Predict & Evaluate CNN Model

In [41]:
y_pred_cnn = np.argmax(cnn_model.predict(X_test_img), axis=1)
y_test_cnn = np.argmax(y_test_img, axis=1)
cnn_report = classification_report(y_test_cnn, y_pred_cnn)
cnn_cm = confusion_matrix(y_test_cnn, y_pred_cnn)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step


--------------------- Save Results to Results.txt ---------------------

In [42]:
with open("Results.txt", "w") as f:
    f.write("=== Feature Extraction Model (Random Forest) Performance ===\n")
    f.write(rf_report + "\n")
    f.write("Confusion Matrix:\n")
    f.write(np.array2string(rf_cm) + "\n")
    f.write("="*50 + "\n\n")

    f.write("=== CNN Model Performance ===\n")
    f.write(cnn_report + "\n")
    f.write("Confusion Matrix:\n")
    f.write(np.array2string(cnn_cm) + "\n")
    f.write("="*50 + "\n\n")

print("Training completed. Results saved to 'Results.txt'.")

Training completed. Results saved to 'Results.txt'.
